##  7. Flickr

In [ ]:
## photo search: https://www.flickr.com/services/api/flickr.photos.search.html
## photo url:  https://www.flickr.com/services/api/misc.urls.html
## flickeapi : https://stuvel.eu/flickrapi-doc/index.html

##  limited to 3600 calls per hour

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
import pandas as pd
import time
import requests
from lxml import etree
import urllib
import os
!pip install xmltodict
import urllib,xmltodict,time,os,os.path
import pprint
!pip install flickrapi
import flickrapi
import json
import xml.etree.ElementTree
print("import finish")


In [ ]:
# detail: https://www.flickr.com/services/api/flickr.photos.search.html

# API_KEY = 'f559f910b851cd1d5f45feeccfb626a9'  # your flickr API_key #https://www.flickr.com/services/apps/create/apply/
# API_SECRET = '5b8da759681d52d7'
API_KEY = 'cca373ce6c54afdad993928d5c035139'  # your flickr API_key #https://www.flickr.com/services/apps/create/apply/
API_SECRET ='56e2a6f6dcbb9f81'

# download images

### (more than 250，less than 4000)

In [ ]:
tags=''
tag_mode = 'any'  # 'all' or 'any'
text = 'sum4'#改！！！！！！！！！！！！！！！！！！！！！！！！！
min_taken_date = '2020-01-01 00:00:00 '
max_taken_date = '2020-11-30 00:00:00'
privacy_filter  = '1'  # 1 public photos
has_geo = '1' # 0: photo that has not been geotagged
radius = 32  # 0-32km
radius_units = 'km'
geo_context = 2  # 0, not defined.  # 1, indoors.  # 2, outdoors.

In [ ]:
# 多伦多（Toronto）
# lat_csv = '43.65107'
# lon_csv = '-79.347015'

# # 温哥华（Vancouver）
# lat_csv = '49.282729'
# lon_csv = '-123.120738'

# 东京
lat_csv = '35.69204'
lon_csv = '139.70043'

In [ ]:
#改csv地址！！！！！！！！！
result_list = []
page_number = 0
count_ID = 0

flickr=flickrapi.FlickrAPI(API_KEY, API_SECRET, cache=True)

while page_number < 17:
    try:
        photos = flickr.photos.search(tags=tags, has_geo=1, tag_mode=tag_mode, lat=lat_csv, lon=lon_csv, radius=radius,
                                      min_taken_date=min_taken_date, max_taken_date=max_taken_date, privacy_filter=1,
                                      extras='geo,url_c,date_upload, date_taken,tags', per_page = '250',page = page_number)
        print('=========','page:',page_number,'total:',len( photos[0]),'=========')
    except Exception as e:
        print('Error')
    for photo in photos[0]:
        print(count_ID)
        #获得照片的url,设置大小为url_c(具体参数请参看FlickrAPI官方文档介绍)
        id = photo.get('id')
        url= photo.get('url_c')
        lat = photo.get('latitude')
        lng = photo.get('longitude')
        date = photo.get('datetaken')
        tag = photo.get('tags')
        result_list.append([count_ID,id,url,lat,lng,date,tag])
        count_ID +=1
    time.sleep(1)
    page_number +=1

df = pd.DataFrame(result_list)
df.columns = ['count_ID','id','url','Lat','Lng','date','tag']
df.to_csv('/content/drive/MyDrive/urbandata/csv_image/csv_TOKYO_official/4' + '.csv',index = False)
df.head(5)

In [ ]:
import folium

map = folium.Map(location=[lat_csv,lon_csv],tiles='Stamen Toner',zoom_start=15)
folium.Marker([lat_csv, lon_csv]).add_to(map)
folium.Circle([lat_csv, lon_csv],radius=500).add_to(map)

for index, row in df.iterrows():
    folium.CircleMarker([row['Lat'], row['Lng']],radius=1,fill=True,opacity=0.8,color='red').add_to(map)

map

In [ ]:
#改image存储地址
for i in range(1663,len(df)):
    count_ID = df.loc[i,'count_ID']
    url = df.loc[i,'url']
    try:
        urllib.request.urlretrieve(url, '/content/drive/MyDrive/urbandata/Fliker/Tokyo/4/' + '_'+ str(count_ID) + '.jpg')
        print('downloading:',text, count_ID)
    except :
        print('could not down load:',text, count_ID)
        continue
    time.sleep(1)

print('finish')